In [ ]:
#bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#arquivos csv
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv('sampleSubmission.csv')

In [ ]:
#primeiras 5 linhas do dado
test.head()

In [ ]:
train.head()

In [ ]:
#somente dados de quantidade das pessoas que alugaram bike
train['count']

In [ ]:
#dados com weather 1
train.loc[train.weather == 1]

In [ ]:
#dados com weather 2
train.loc[train.weather == 2]

In [ ]:
#dados com weather 3
train.loc[train.weather == 3]

In [ ]:
#dados com weather 4
train.loc[train.weather == 4]

In [ ]:
#pegando somente os horarios de datetime
train['time'] = train['datetime'].str[10:13]
train['time']


In [ ]:
test['time'] = test['datetime'].str[10:13]
test['time']

In [ ]:
#transformando dtype: object para dtype: int
train['time'] = train['time'].astype(int)
train["time"]

In [ ]:
test['time'] = test['time'].astype(int)
test["time"]

In [ ]:
#ver as relações entre os dados
correlacao = train.corr()
plt.figure()
sns.heatmap(data=correlacao, annot=True)

In [83]:
#coluna de previsão é y
y = test.count

In [85]:
#recursos
test_features = ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'time']

In [86]:
#coluna de recursos é x
x = test[test_features]

In [91]:
x.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,time
count,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000
mean,2.493300,0.029108,0.685815,1.436778,20.620607,24.012865,64.125212,12.631157,11.555367
std,1.091258,0.168123,0.464226,0.648390,8.059583,8.782741,19.293391,8.250151,6.912526
min,1.000000,0.000000,0.000000,1.000000,0.820000,0.000000,16.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,13.940000,16.665000,49.000000,7.001500,6.000000
50%,3.000000,0.000000,1.000000,1.000000,21.320000,25.000000,65.000000,11.001400,12.000000
75%,3.000000,0.000000,1.000000,2.000000,27.060000,31.060000,81.000000,16.997900,18.000000
max,4.000000,1.000000,1.000000,4.000000,40.180000,50.000000,100.000000,55.998600,23.000000


In [92]:
x.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,time
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,4


In [94]:
from sklearn.tree import DecisionTreeRegressor